In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, auc, mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

import numpy as np

pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df = pd.read_csv('modified_train.csv', index_col=0)

In [ ]:
df.columns

In [ ]:
df[df.toxicity_b == 0].shape

In [ ]:
df[df.toxicity_b == 1].shape


In [ ]:
print('Процентное соотношение значений ' + str(df['toxicity_b'].value_counts(normalize=True) * 100))

In [ ]:
df['ctws'].fillna('', inplace=True)

# Одноклассовая класификация

Обучение моделей

In [ ]:
X = df['ctws']
y = df['toxicity_b']

Параметр stratify используется для того, чтобы гарантировать, что распределение целевой переменной y (или другого указанного массива) в выборках train и test будет аналогичным.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Функция обучения модели и предсказания и визуализации результатов
def train_and_predict_toxicity(model, X_train, X_test, y_train):
    #Обучение модели
    model.fit(X_train, y_train)

    # Предсказание классов
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Предсказание вероятностей классов (если доступно)
    if hasattr(model, 'predict_proba'):
        # Вероятности для положительного класса
        y_pred_proba_train = model.predict_proba(X_train)[:, 1]
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
    else:
        y_pred_proba_train = None
        y_pred_proba_test = None

    return y_pred_train, y_pred_test, y_pred_proba_train, y_pred_proba_test


# Функция для отображения метрик
def show_metrics(y_train, y_test, y_pred_train, y_pred_test, y_pred_proba_train, y_pred_proba_test, title):
    # Вычисляем точность (precision) — доля истинных положительных среди всех предсказанных положительных
    # одна из метрик, оценивающих качество решения задачи бинарной классификации в машинном обучении. 
    # Она давно себя зарекомендовала: это одно интерпретируемое и легко вычисляемое число, оценивающее качество алгоритма
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)

    # Вычисляем полноту (recall) — доля истинных положительных среди всех фактических положительных
    recall_train = recall_score(y_train, y_pred_train)
    recall_test = recall_score(y_test, y_pred_test)

    # Вычисляем F1-метрику — гармоническое среднее между точностью и полнотой
    f1_train = f1_score(y_train, y_pred_train)
    f1_test = f1_score(y_test, y_pred_test)
    
    # посчитать MSE, MAE, RMSE

    # Вычисляем среднеквадратичную ошибку (MSE) — среднее значение квадратов разностей между фактическими и предсказанными значениями
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)

    # Вычисляем среднюю абсолютную ошибку (MAE) — среднее значение абсолютных разностей между фактическими и предсказанными значениями
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)

    # Вычисляем квадратный корень из MSE (RMSE) — интерпретация MSE в тех же единицах, что и данные
    rmse_train = np.sqrt(mse_train)
    rmse_test = np.sqrt(mse_test)
    

    print(title + '\n')

    print('Precision\t\tRecall\t\t\tF1\n')
    print(f'Train: {precision_train:.2f}\t\tTrain: {recall_train:.2f}\t\tTrain: {f1_train:.2f}')
    print(f'Test: {precision_test:.2f}\t\tTest: {recall_test:.2f}\t\tTest: {f1_test:.2f}')


    # Выводим таблицу с метриками MSE, MAE и RMSE
    print('\nMSE\t\t\tMAE\t\t\tRMSE\n')
    print(f'Train: {mse_train:.2f}\tTrain: {mae_train:.2f}\tTrain: {rmse_train:.2f}')
    print(f'Test: {mse_test:.2f}\tTest: {mae_test:.2f}\tTest: {rmse_test:.2f}')

    def show_confusion_matrix(y_true, y_pred, sample_name, ax):
        # Вычисляем матрицу ошибок (confusion matrix) с нормализацией по всем элементам
        # Это показывает пропорции вместо абсолютных значений
        cm = confusion_matrix(y_true, y_pred, normalize='all')

        # Определяем метки классов для отображения в матрице
        labels = ['Non-Toxic (0)', 'Toxic (1)']
        # Преобразуем матрицу ошибок в DataFrame для удобной визуализации
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)

        # Создаем тепловую карту (heatmap) на основе матрицы ошибок    
        sns.heatmap(cm_df, annot=True, fmt=".2%", cmap='twilight_shifted', ax=ax)
        ax.set_title(f'Confusion matrix ({sample_name})')
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')

    def show_roc_curve(y_true, y_pred_proba, model_name, sample_name, ax):
        # Вычисляем значения FPR (False Positive Rate) и TPR (True Positive Rate)
        # на различных порогах классификации
        fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
        # Вычисляем площадь под кривой (AUC, Area Under Curve)
        auc = roc_auc_score(y_true, y_pred_proba)
        
        # Строим ROC-кривую, отображая FPR по оси X, а TPR по оси Y
        ax.plot(fpr, tpr)
        # Добавляем диагональную линию (y = x) для ориентира (случайная модель)
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_xlabel('FPR')
        ax.set_ylabel('TPR')
        ax.set_title(f'{model_name} ({sample_name}) ROC (AUC = {auc:.2f})')
        # Включаем сетку для удобства чтения графика
        ax.grid(True)

    # Строит кривую Precision-Recall (точность-полнота).
    def show_pr_curve(y_true, y_pred_proba, model_name, sample_name, ax):
        # Вычисляем значения precision (точности) и recall (полноты) для различных порогов
        precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
        # Вычисляем площадь под кривой Precision-Recall (AUC-PR)
        auc_pr = auc(recall, precision)

        # Строим график кривой Precision-Recall
        ax.plot(recall, precision)
        ax.set_xlabel('Recall')
        ax.set_ylabel('Precision')
        ax.set_title(f'{model_name} ({sample_name}) Precision-Recall Curve (AUC = {auc_pr:.2f})')
        ax.grid(True)

    # Проверяем, есть ли предсказания вероятностей для тренировочной и тестовой выборок
    if y_pred_proba_train is not None and y_pred_proba_test is not None:
        _, axs = plt.subplots(3, 2, figsize=(16, 18))

        # Показываем матрицу ошибок для тренировочной и тестовой выборок
        show_confusion_matrix(y_train, y_pred_train, 'Train', axs[0, 0])
        show_confusion_matrix(y_test, y_pred_test, 'Test', axs[0, 1])

        # Показываем ROC-кривые для тренировочной и тестовой выборок
        show_roc_curve(y_train, y_pred_proba_train, title, 'Train', axs[1, 0])
        show_roc_curve(y_test, y_pred_proba_test, title, 'Test', axs[1, 1])

        # Показываем кривые Precision-Recall для тренировочной и тестовой выборок
        show_pr_curve(y_train, y_pred_proba_train, title, 'Train', axs[2, 0])
        show_pr_curve(y_test, y_pred_proba_test, title, 'Test', axs[2, 1])
    else:
        # Если предсказаний вероятностей нет, создаем сетку из 1 строки и 2 столбцов (2 графика)
        _, axs = plt.subplots(1, 2, figsize=(14, 6))

        show_confusion_matrix(y_train, y_pred_train, 'Train', axs[0])
        show_confusion_matrix(y_test, y_pred_test, 'Test', axs[1])

    plt.tight_layout()
    plt.show()


# Функция получения мешка слов
def get_bows(X_train, X_test):
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(X_train), vectorizer.transform(X_test)


# Векторизация  TF-IDF
def get_tfidf(X_train, X_test):
    vectorizer = TfidfVectorizer()
    return vectorizer.fit_transform(X_train), vectorizer.transform(X_test)

# Обучение с мешком слов

In [ ]:
X_train_bow, X_test_bow = get_bows(X_train, X_test)

# Логистическая регрессия


In [ ]:
(
    y_log_reg_bow_pred_train,
    y_log_reg_bow_pred_test,
    y_log_reg_bow_pred_proba_train,
    y_log_reg_bow_pred_proba_test
) = train_and_predict_toxicity(
    model=LogisticRegression(max_iter=1000),
    X_train=X_train_bow,
    X_test=X_test_bow,
    y_train=y_train,
)

In [ ]:
show_metrics(
    y_train=y_train,
    y_test=y_test,
    y_pred_train=y_log_reg_bow_pred_train,
    y_pred_test=y_log_reg_bow_pred_test,
    y_pred_proba_train=y_log_reg_bow_pred_proba_train,
    y_pred_proba_test=y_log_reg_bow_pred_proba_test,
    title='LogisticRegression BoW'
)

# SVM

In [ ]:
(
    y_linear_svc_bow_pred_train,
    y_linear_svc_bow_pred_test,
    y_linear_svc_bow_pred_proba_train,
    y_linear_svc_bow_pred_proba_test
) = train_and_predict_toxicity(
    model=LinearSVC(dual=False),
    X_train=X_train_bow,
    X_test=X_test_bow,
    y_train=y_train,
)

In [41]:
show_metrics(
    y_train=y_train,
    y_test=y_test,
    y_pred_train=y_linear_svc_bow_pred_train,
    y_pred_test=y_linear_svc_bow_pred_test,
    y_pred_proba_train=y_linear_svc_bow_pred_proba_train,
    y_pred_proba_test=y_linear_svc_bow_pred_proba_test,
    title='SVM BoW'
)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

# Наивный Байес (Наивный байесовский классификатор)


In [ ]:
(
    y_nb_bow_pred_train,
    y_nb_bow_pred_test,
    y_nb_bow_pred_proba_train,
    y_nb_bow_pred_proba_test
) = train_and_predict_toxicity(
    model=MultinomialNB(),
    X_train=X_train_bow,
    X_test=X_test_bow,
    y_train=y_train,
)

In [ ]:
show_metrics(
    y_train=y_train,
    y_test=y_test,
    y_pred_train=y_nb_bow_pred_train,
    y_pred_test=y_nb_bow_pred_test,
    y_pred_proba_train=y_nb_bow_pred_proba_train,
    y_pred_proba_test=y_nb_bow_pred_proba_test,
    title='MultinomialNB BoW'
)

# Обучение с TF-IDF

In [ ]:
X_train_tfidf, X_test_tfidf = get_tfidf(X_train, X_test)

Логистическая регрессия

In [ ]:
(
    y_log_reg_tfidf_pred_train,
    y_log_reg_tfidf_pred_test,
    y_log_reg_tfidf_pred_proba_train,
    y_log_reg_tfidf_pred_proba_test
) = train_and_predict_toxicity(
    model=LogisticRegression(max_iter=1000),
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
)

In [ ]:
show_metrics(
    y_train=y_train,
    y_test=y_test,
    y_pred_train=y_log_reg_tfidf_pred_train,
    y_pred_test=y_log_reg_tfidf_pred_test,
    y_pred_proba_train=y_log_reg_tfidf_pred_proba_train,
    y_pred_proba_test=y_log_reg_tfidf_pred_proba_test,
    title='LogisticRegression TF-IDF'
)

### SVM

In [ ]:
(
    y_linear_svc_tfidf_pred_train,
    y_linear_svc_tfidf_pred_test,
    y_linear_svc_tfidf_pred_proba_train,
    y_linear_svc_tfidf_pred_proba_test
) = train_and_predict_toxicity(
    model=LinearSVC(dual=False),
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
)

In [ ]:
show_metrics(
    y_train=y_train,
    y_test=y_test,
    y_pred_train=y_linear_svc_tfidf_pred_train,
    y_pred_test=y_linear_svc_tfidf_pred_test,
    y_pred_proba_train=y_linear_svc_tfidf_pred_proba_train,
    y_pred_proba_test=y_linear_svc_tfidf_pred_proba_test,
    title='SVM TF-IDF'
)

### Наивный Байес


In [ ]:
(
    y_nb_tfidf_pred_train,
    y_nb_tfidf_pred_test,
    y_nb_tfidf_pred_proba_train,
    y_nb_tfidf_pred_proba_test
) = train_and_predict_toxicity(
    model=MultinomialNB(),
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
)

In [ ]:
show_metrics(
    y_train=y_train,
    y_test=y_test,
    y_pred_train=y_nb_tfidf_pred_train,
    y_pred_test=y_nb_tfidf_pred_test,
    y_pred_proba_train=y_nb_tfidf_pred_proba_train,
    y_pred_proba_test=y_nb_tfidf_pred_proba_test,
    title='MultinomialNB TF-IDF'
)

# Multi-Label классификация


## Обучение моделей


In [42]:
labels = df[['toxicity_b', 'severe_toxicity_b', 'obscene_b', 'identity_attack_b', 'insult_b', 'threat_b', 'sexual_explicit_b']]

X = df['ctws']
y = labels

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

def train_and_predict_toxicity_multilabel(model, X_train, X_test, y_train):
    ovr = OneVsRestClassifier(model, n_jobs=-1)
    ovr.fit(X_train, y_train)

    y_pred_train = ovr.predict(X_train)
    y_pred_test = ovr.predict(X_test)

    return y_pred_train, y_pred_test

def show_classification_report(y_train, y_test, y_pred_train, y_pred_test, target_names, title):
    report_train = classification_report(y_train, y_pred_train, target_names=target_names, zero_division=0, output_dict=True)
    report_test = classification_report(y_test, y_pred_test, target_names=target_names, zero_division=0, output_dict=True)

    report_train_df = pd.DataFrame(report_train).transpose()
    report_test_df = pd.DataFrame(report_test).transpose()

    print(f"{title}\n")

    print("METRICS PER CLASS\n")
    print("Train:")
    display(report_train_df.iloc[:-4, :])
    print("\nTest:")
    display(report_test_df.iloc[:-4, :])

    print("\nOVERALL METRICS\n")
    print("Train:")
    display(report_train_df.loc[["micro avg", "macro avg", "weighted avg", "samples avg"]].drop(columns=['support']))
    print("\nTest:")
    display(report_test_df.loc[["micro avg", "macro avg", "weighted avg", "samples avg"]].drop(columns=['support']))

    def show_heatmap(report_df, ax, model_name, sample_name):
        heatmap_df = report_df.iloc[:-4, :]

        sns.heatmap(heatmap_df.iloc[:, :-1], annot=True, cmap="Blues", fmt=".2f", cbar=True, linewidths=0.5, ax=ax)
        ax.set_title(f"{model_name} Classification Report Heatmap ({sample_name})", fontsize=16)
        ax.set_ylabel("Classes", fontsize=12)
        ax.set_xlabel("Metrics", fontsize=12)
        ax.tick_params(axis='x', rotation=45)
        ax.tick_params(axis='y', rotation=0)

    _, axes = plt.subplots(1, 2, figsize=(20, 8))

    show_heatmap(report_train_df, axes[0], title, "Train")
    show_heatmap(report_test_df, axes[1], title, "Test")

    plt.tight_layout()
    plt.show()

# Обучение с мешком слов


In [ ]:
X_train_bow, X_test_bow = get_bows(X_train, X_test)

### Логистическая регрессия

In [ ]:
y_log_reg_bow_pred_train, y_log_reg_bow_pred_test = train_and_predict_toxicity_multilabel(
    model=LogisticRegression(max_iter=1000),
    X_train=X_train_bow,
    X_test=X_test_bow,
    y_train=y_train,
)

In [ ]:
show_classification_report(
    y_train,
    y_test,
    y_log_reg_bow_pred_train,
    y_log_reg_bow_pred_test,
    y.columns,
    'Multi-Label LogisticRegression BoW'
)

### SVM

In [ ]:
y_svm_bow_pred_train, y_svm_bow_pred_test = train_and_predict_toxicity_multilabel(
    model=LinearSVC(dual=False),
    X_train=X_train_bow,
    X_test=X_test_bow,
    y_train=y_train,
)

In [ ]:
show_classification_report(
    y_train,
    y_test,
    y_svm_bow_pred_train,
    y_svm_bow_pred_test,
    y.columns,
    'Multi-Label SVM BoW'
)

### Наивный Байес

In [ ]:
y_nb_bow_pred_train, y_nb_bow_pred_test = train_and_predict_toxicity_multilabel(
    model=MultinomialNB(),
    X_train=X_train_bow,
    X_test=X_test_bow,
    y_train=y_train,
)

In [ ]:
show_classification_report(
    y_train,
    y_test,
    y_nb_bow_pred_train,
    y_nb_bow_pred_test,
    y.columns,
    'Multi-Label MultinomialNB BoW'
)

# Обучение с TF-IDF

In [44]:
X_train_tfidf, X_test_tfidf = get_tfidf(X_train, X_test)

Логистическая регрессия


In [ ]:
y_log_reg_tfidf_pred_train, y_log_reg_tfidf_pred_test = train_and_predict_toxicity_multilabel(
    model=LogisticRegression(max_iter=1000),
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
)

In [ ]:
show_classification_report(
    y_train,
    y_test,
    y_log_reg_tfidf_pred_train,
    y_log_reg_tfidf_pred_test,
    y.columns,
    'Multi-Label LogisticRegression TF-IDF'
)

### SVM

In [ ]:
y_svm_tfidf_pred_train, y_svm_tfidf_pred_test = train_and_predict_toxicity_multilabel(
    model=LinearSVC(dual=False),
    X_train=X_train_tfidf,
    X_test=X_test_tfidf,
    y_train=y_train,
)

In [ ]:
show_classification_report(
    y_train,
    y_test,
    y_svm_tfidf_pred_train,
    y_svm_tfidf_pred_test,
    y.columns,
    'Multi-Label SVM TF-IDF'
)

Наивный Байес
